In [1]:
import scipy
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
import cmocean
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [2]:
def convertDiffuseFile(fileInName):   
    fileContents = scipy.io.loadmat(fileInName)
    covisDict = {}
    covisDict['header'] = fileContents['__header__'].decode('utf-8')
    covisDict['version'] = fileContents['__version__']
    covisDict['globals'] = fileContents['__globals__']
    covis = fileContents['covis'][0][0]
    for name in covis.dtype.names:
        covisDict[name] = covis[name]
    
    print(f'{fileInName} has been converted')
    return covisDict

In [3]:
def convertImagingFile(fileInName):
    fileContents = scipy.io.loadmat(fileInName)
    covisDict = {}
    covisDict['header'] = fileContents['__header__'].decode('utf-8')
    covisDict['version'] = fileContents['__version__']
    covisDict['globals'] = fileContents['__globals__']
    covis = fileContents['imaging'][0][0]
    for name in covis.dtype.names:
        covisDict[name] = covis[name]
    
    print(f'{fileInName} has been converted')
    return covisDict

In [4]:
def readCoords2D(fileInName):
    covisDict = convertDiffuseFile(fileInName)
    wList = [w for w in covisDict['grid'][0][0][0]['w']]
    xList = [x for x in covisDict['grid'][0][0][0]['x']]
    yList = [y for y in covisDict['grid'][0][0][0]['y']]
    vList = [v for v in covisDict['grid'][0][0][0]['v']]
    coordsDict = {'xList': xList, 'yList': yList, 'vList': vList, 'wList': wList}
    return coordsDict

In [6]:
def readCoords3D(fileInName, dataVal):
    covisDict = convertImagingFile(fileInName)
    xList = [x for x in covisDict['grid'][0][0]['x']]
    yList = [y for y in covisDict['grid'][0][0]['y']]
    zList = [z for z in covisDict['grid'][0][0]['z']]
    vList = [v for v in covisDict['grid'][0][0][f'{dataVal}']]
    wList = [w for w in covisDict['grid'][0][0]['w']]
    coordsDict = {'xList': xList, 'yList': yList, 'zList': zList, 'vList': vList, 'wList': wList}
    return coordsDict

In [5]:
def readCoordsBathymetry(bathyFileInName, diffuseFileInName):
    bathyDict = convertDiffuseFile(bathyFileInName)
    diffuseDict = convertDiffuseFile(diffuseFileInName)
    xList = [x for x in bathyDict['grid'][0][0]['x']]
    yList = [y for y in bathyDict['grid'][0][0]['y']]
    zList = [z for z in bathyDict['grid'][0][0]['v']]
    vList = [v for v in diffuseDict['grid'][0][0][0]['v']]
    coordsDict = {'xList': xList, 'yList': yList, 'zList': zList, 'vList': vList}
    return coordsDict

In [7]:
def removeZeroSlices(arr, axis, coordsDict):
    red_axes = tuple(i for i in range(arr.ndim) if i != axis)
    mask = np.any(arr, axis = red_axes)
    slicing = tuple(slice(None) if i != axis else mask for i in range(arr.ndim))
    for key, value in coordsDict.items():
        coordsDict[key] = value[slicing]
    return coordsDict

In [8]:
def removeAllZeros(coordsDict):
    mask = coordsDict['vList'] > 0
    coordsDict = {key: coordsDict[key][mask] for key in coordsDict.keys()}
    return coordsDict

In [9]:
def createNDArray(coordsDict, dim):
    xmin, xmax = coordsDict['xList'].min(), coordsDict['xList'].max()
    ymin, ymax = coordsDict['yList'].min(), coordsDict['yList'].max()
    zmin, zmax = 0, 0
    minmaxDict = {'x': (xmin, xmax), 'y': (ymin, ymax)}

    if dim == 3:
        zmin, zmax = coordsDict['zList'].min(), coordsDict['zList'].max()
        minmaxDict['z'] = (zmin, zmax)
    
    dimDict = {}
    for key, value in minmaxDict.items():
        dimDict[key] = int((value[1] - value[0]) * 4 + 1)

    if dim == 2:
        for key, value in coordsDict.items():
            coordsDict[key] = value.reshape(dimDict['x'], dimDict['y'])
    elif dim == 3:
        for key, value in coordsDict.items():
            coordsDict[key] = value.reshape(dimDict['x'], dimDict['y'], dimDict['z'])

    return coordsDict

In [10]:
def createCoordsOfInterest(fileInName, opt1, opt2 = 'Ia', bathyFile = r'matFiles\covis_bathy_2019b.mat'):
    if opt1 == 2:
        coordsDict = readCoords2D(fileInName)
        coordsDict = {key: np.array(coordsDict[key]) for key in coordsDict.keys()}

        vList = coordsDict['vList']
        for i in [0, 1, 2]:
            coordsDict = removeZeroSlices(vList, i, coordsDict)

        coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}

    elif opt1 == 3:
        coordsDict = readCoords3D(fileInName, opt2)
        coordsDict = {key: np.array(coordsDict[key]) for key in coordsDict.keys()}

        vList = coordsDict['vList']
        for i in [0, 1, 2]:
            coordsDict = removeZeroSlices(vList, i, coordsDict)

        coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}

    elif opt1 == 4:
        coordsDict = readCoordsBathymetry(bathyFile, fileInName)
        coordsDict = {key: np.array(coordsDict[key]) for key in coordsDict.keys()}
        coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}

        coordsDict['vList'] = coordsDict['vList'].reshape(121, 121)[:,:101]
        coordsDict['vList'] = np.concatenate(coordsDict['vList']).flatten()
        coordsDict['zList'] = np.nan_to_num(coordsDict['zList'])
        
        x = coordsDict['xList']
        y = coordsDict['yList']
        indices = np.lexsort((y, x))
        coordsDict = {key: coordsDict[key][indices].reshape(101, 121) for key in coordsDict.keys()}
    
    return coordsDict

In [11]:
def plotDiffuse2DV1(fileInName):
    coordsOfInterest = createCoordsOfInterest(fileInName, 2)
    plt.figure(figsize=(8, 6))

    coordsOfInterest = removeAllZeros(coordsOfInterest)
    coordsOfInterest['vList'] = np.log10(coordsOfInterest['vList'])
    max = np.floor(np.amax(coordsOfInterest['vList']))
    min = np.ceil(np.amin(coordsOfInterest['vList']))
    scatter = plt.scatter(coordsOfInterest['xList'], coordsOfInterest['yList'], c = coordsOfInterest['vList'], cmap = 'viridis')

    cbar = plt.colorbar(scatter)
    cbar_ticks = np.arange(min, max + 1, 1)
    cbar.set_ticks(cbar_ticks)
    cbar.set_label('Data Values')

    plt.xlabel('East of COVIS (m)')
    plt.ylabel('North of COVIS (m)')
    plt.title(fileInName.split('\\')[-1])
    plt.show()


In [12]:
def plotFullImaging3DV1(fileInName, opt2):
    coordsOfInterest = createCoordsOfInterest(fileInName, 3, opt2)
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    coordsOfInterest = removeAllZeros(coordsOfInterest)
    coordsOfInterest['vList'] = np.log10(coordsOfInterest['vList'])
    max = np.floor(np.amax(coordsOfInterest['vList']))
    min = np.ceil(np.amin(coordsOfInterest['vList']))
    scatter = ax.scatter(coordsOfInterest['xList'], coordsOfInterest['yList'], coordsOfInterest['zList'], c = coordsOfInterest['vList'], cmap = 'viridis')
    
    cbar = plt.colorbar(scatter)
    cbar.set_label(f'{opt2} Values')
    cbar_ticks = np.arange(min, max + 1, 1)
    cbar.set_ticks(cbar_ticks)

    ax.set_xlabel('East of COVIS (m)')
    ax.set_ylabel('North of COVIS (m)')
    ax.set_zlabel('Z Axis')
    ax.set_title(f'3D Scatter Plot (Logarithmic scale, {opt2})')

    plt.show()

In [13]:
def plotFullImaging3DV2(fileInName, opt2):
	coordsOfInterest = createCoordsOfInterest(fileInName, 3, opt2)
	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']
	z = coordsOfInterest['zList']
	values = coordsOfInterest['vList']
	
	values = values.flatten()
	mask = values != 0
	values[mask] = np.log10(values[mask])
	min = np.min(values[values != 0])
	max = np.max(values[values != 0])
	
	# values = values.flatten()
	# values[values == 0] = 10e-10
	# values = np.log10(values)
	# min = np.min(values[values != 0])
	# max = np.max(values[values != 0])

	fig = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'Inferno',
		isomin = min,
		isomax = max,
		surface_count = 2,
		opacity = .5,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))
	
	fig.update_layout(
		title=f'3D Isosurface Plot (Logarithmic scale, {opt2})',
		scene=dict(
			xaxis_title='East of COVIS (m)',
			yaxis_title='North of COVIS (m)',
			zaxis_title='Z Axis'
		),
		coloraxis_colorbar=dict(
			title=f'{opt2} Values',
			tickvals=np.linspace(min, max, num=5),
			ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)
	
	fig.show()


In [ ]:
plotFullImaging3DV2(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Ia_filt')

In [15]:
def plotFullImagingInferno(fileInName, opt2):
	coordsOfInterest = createCoordsOfInterest(fileInName, 3, opt2)
	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	for key, value in coordsOfInterest.items():
		coordsOfInterest[key] = value.flatten()

	xmask = (coordsOfInterest['xList'] >= -20) & (coordsOfInterest['xList'] <= 0)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[xmask]
	ymask = (coordsOfInterest['yList'] >= -10) & (coordsOfInterest['yList'] <= 10)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[ymask]
	zmask = (coordsOfInterest['zList'] >= 0) & (coordsOfInterest['zList'] <= 20)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[zmask]

	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']
	z = coordsOfInterest['zList']
	values = coordsOfInterest['vList']
	values[values == 0] = 10e-10
	values = np.log10(values)
	min = -1
	max = -9

	fig = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'thermal',
		isomin = min,
		isomax = max,
		surface_count = 10,
		opacity = 0.1,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	fig.update_layout(
		title=f'Inferno (Logarithmic scale, {opt2})',
		scene=dict(
				xaxis_title='East of COVIS (m)',
				yaxis_title='North of COVIS (m)',
				zaxis_title='Z Axis'
		),
		coloraxis_colorbar=dict(
				title=f'{opt2} Values',
				tickvals=np.linspace(min, max, num=5),
				ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)

	fig.show()

In [ ]:
plotFullImagingInferno(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Ia_filt')

In [16]:
def plotFullImagingMushroom(fileInName, opt2):
	coordsOfInterest = createCoordsOfInterest(fileInName, 3, opt2)
	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	for key, value in coordsOfInterest.items():
		coordsOfInterest[key] = value.flatten()

	xmask = (coordsOfInterest['xList'] >= -10) & (coordsOfInterest['xList'] <= 0)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[xmask]
	ymask = (coordsOfInterest['yList'] >= 0) & (coordsOfInterest['yList'] <= 10)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[ymask]
	zmask = (coordsOfInterest['zList'] >= 0) & (coordsOfInterest['zList'] <= 15)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[zmask]

	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']
	z = coordsOfInterest['zList']
	values = coordsOfInterest['vList']
	values[values == 0] = 10e-10
	values = np.log10(values)
	
	fig = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'Inferno',
		isomin = -1,
		isomax = -9,
		surface_count = 10,
		opacity = .1,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	fig.update_layout(
		title=f'Mushroom (Logarithmic scale, {opt2})',
		scene=dict(
				xaxis_title='East of COVIS (m)',
				yaxis_title='North of COVIS (m)',
				zaxis_title='Z Axis'
		),
		coloraxis_colorbar=dict(
				title=f'{opt2} Values',
				tickvals=np.linspace(min, max, num=5),
				ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)

	fig.show()

In [ ]:
plotFullImagingMushroom(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Ia_filt')

In [17]:
def plotDiffuseBathymetryV1(fileInName, bathyFile = r'matFiles\covis_bathy_2019b.mat'):
    coordsOfInterest = createCoordsOfInterest(fileInName, 4, bathyFile)
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    coordsOfInterest = removeAllZeros(coordsOfInterest)
    coordsOfInterest['vList'] = np.log10(coordsOfInterest['vList'])
    max = np.floor(np.amax(coordsOfInterest['vList']))
    min = np.ceil(np.amin(coordsOfInterest['vList']))

    scatter = ax.scatter(coordsOfInterest['xList'], coordsOfInterest['yList'], coordsOfInterest['zList'], c = coordsOfInterest['vList'], cmap = 'cmo.thermal')

    cbar = plt.colorbar(scatter)
    cbar.set_label('Values')
    cbar_ticks = np.arange(min, max + 1)
    cbar.set_ticks(cbar_ticks)

    ax.set_xlabel('East of COVIS (m)')
    ax.set_ylabel('North of COVIS (m)')
    ax.set_zlabel('Depth Relative to COVIS (m)')
    ax.set_title(f'Diffuse on Bathymetry')
    
    plt.show()

In [18]:
def plotDiffuseBathymetryV2(fileInName, opt2, bathyFile = r'matFiles\covis_bathy_2019b.mat'):
	coordsOfInterest = createCoordsOfInterest(fileInName, 4, bathyFile)

	z = coordsOfInterest['zList']
	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']

	xmin, xmax = -20, 20
	ymin, ymax = -30, 1
	zmin, zmax = z.min(), z.max()

	sh_0, sh_1 = z.shape
	x = np.linspace(x.min(), x.max(), sh_0)
	y = np.linspace(y.min(), y.max(), sh_1)
	xrange = x.max() - x.min()
	range_y = y.max() - y.min()
	range_z = z.max() - z.min()
	maxrange = np.max([xrange, range_y, range_z])

	fig = go.Figure(data=[go.Surface(
		z=z,
		x=x,
		y=y, 
		)]
	)

	fig.update_layout(
		title='Bathymetry', 
		scene=dict(
			aspectmode="manual",
			aspectratio=dict(
				x=xrange / maxrange,
				y=range_y / maxrange,
				z=range_z / maxrange
			),
			xaxis=dict(range=[xmin, xmax]),
			yaxis=dict(range=[ymin, ymax]),
			zaxis=dict(range=[zmin, zmax])
		)
	)

	fig.show()

In [19]:
def plotDiffuseBathymetryV3(fileInName, opt2, bathyFile = r'matFiles\covis_bathy_2019b.mat'):
    coordsOfInterest = createCoordsOfInterest(fileInName, 4, bathyFile)
    plt.figure(figsize=(12, 8))

    x = coordsOfInterest['xList']
    y = coordsOfInterest['yList']
    data = coordsOfInterest[opt2]
    
    if opt2 == 'zList':
        param = 'height'
    elif opt2 == 'vList':
        param = 'data values'
        data = np.ma.masked_where(data == 0, data)
        mask = data > 0
        data[mask] = np.log10(data[mask])
    
    plt.pcolormesh(x, y, data, shading = 'auto')
    plt.colorbar(label = param)

    plt.xlabel('East of COVIS (m)')
    plt.ylabel('North of COVIS (m)')
    plt.title(f'Pcolor Plot of Bathymetry, {param}')

    plt.show()

In [20]:
def plotFull(diffuseFileName, bathyFileName, imagingFileName, opt2):
	coordsOfInterest = createCoordsOfInterest(diffuseFileName, 4, bathyFileName)

	z = coordsOfInterest['zList']
	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']

	sh_0, sh_1 = z.shape
	x = np.linspace(x.min(), x.max(), sh_0)
	y = np.linspace(y.min(), y.max(), sh_1)
	xrange = x.max() - x.min()
	range_y = y.max() - y.min()
	range_z = z.max() - z.min()
	x = x - 13
	y = y + 13

	fig1 = go.Figure(data=[go.Surface(
		z = z,
		x = x,
		y = y, 
		colorscale = [[0, 'tan'], [1, 'tan']], 
		showscale = False,
		contours = {
            "z": {"show": True, "highlight": True, "highlightcolor": "black", "highlightwidth": 2}
		}
		)]
	)
	
	coordsOfInterest = createCoordsOfInterest(imagingFileName, 3, opt2)
	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	for key, value in coordsOfInterest.items():
		coordsOfInterest[key] = value.flatten()

	xmask = (coordsOfInterest['xList'] >= -20) & (coordsOfInterest['xList'] <= 0)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[xmask]
	ymask = (coordsOfInterest['yList'] >= -10) & (coordsOfInterest['yList'] <= 10)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[ymask]
	zmask = (coordsOfInterest['zList'] >= 0) & (coordsOfInterest['zList'] <= 20)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[zmask]

	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']
	z = coordsOfInterest['zList']
	values = coordsOfInterest['vList']
	values[values == 0] = 10e-10
	values = np.log10(values)
	min = -1
	max = -9

	fig2 = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'thermal',
		isomin = min,
		isomax = max,
		surface_count = 5,
		opacity = 0.2,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	fig2.update_layout(
		coloraxis_colorbar=dict(
			title=f'{opt2} Values',
			x = 1.1,
			tickvals=np.linspace(min, max, num=5),
			ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)

	coordsOfInterest = createCoordsOfInterest(imagingFileName, 3, opt2)
	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	for key, value in coordsOfInterest.items():
		coordsOfInterest[key] = value.flatten()

	xmask = (coordsOfInterest['xList'] >= -10) & (coordsOfInterest['xList'] <= 0)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[xmask]
	ymask = (coordsOfInterest['yList'] >= 0) & (coordsOfInterest['yList'] <= 10)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[ymask]
	zmask = (coordsOfInterest['zList'] >= 0) & (coordsOfInterest['zList'] <= 15)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[zmask]

	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']
	z = coordsOfInterest['zList']
	values = coordsOfInterest['vList']
	values[values == 0] = 10e-10
	values = np.log10(values)
	
	fig3 = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'Inferno',
		isomin = -1,
		isomax = -9,
		surface_count = 5,
		opacity = .2,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	fig3.update_layout(
		coloraxis_colorbar=dict(
			title=f'{opt2} Values',
			x = 1.2,
			tickvals=np.linspace(min, max, num=5),
			ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)

	figFinal = go.Figure(data = fig1.data + fig2.data + fig3.data)
	figFinal.update_layout(
		title=f'Plumes on Bathymetry (Logarithmic scale, {opt2})',
		scene=dict(
			xaxis_title='East of COVIS (m)',
			yaxis_title='North of COVIS (m)',
			zaxis_title='Z Axis',
			xaxis=dict(range=[-20 - 13, 20 - 13]),
			yaxis=dict(range=[-30 + 13, 1 + 13]),
			zaxis=dict(range=[-4, 20])
		)
	)
	
	figFinal.data[1].colorbar.update(title='Inferno', x=1.1)
	figFinal.data[2].colorbar.update(title='Mushroom', x=1.2)

	figFinal.show()

In [ ]:
plotFull(r'matFiles\COVIS-20230701T003002-diffuse1.mat', r'matFiles\covis_bathy_2019b.mat', r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Ia_filt')

In [21]:
def splatAlongAxis(fileInName, opt2, blur, axis = 'x'):
    coordsOfInterest = createCoordsOfInterest(fileInName, 3, opt2)
    coordsOfInterest = removeAllZeros(coordsOfInterest)

    if axis == 'x':
        proj_coords = (coordsOfInterest['yList'], coordsOfInterest['zList'])
        xlabel, ylabel = 'y', 'z'
    elif axis == 'y':
        proj_coords = (coordsOfInterest['xList'], coordsOfInterest['zList'])
        xlabel, ylabel = 'x', 'z'
    elif axis == 'z':
        proj_coords = (coordsOfInterest['xList'], coordsOfInterest['yList'])
        xlabel, ylabel = 'x', 'y'
    
    coordsOfInterest['vList'] = np.log10(coordsOfInterest['vList'])

    minCoord1 = np.min(proj_coords[0])
    maxCoord1 = np.max(proj_coords[0])
    minCoord2 = np.min(proj_coords[1])
    maxCoord2 = np.max(proj_coords[1])
    
    gridSize = 500
    grid, _, _ = np.histogram2d(proj_coords[0], proj_coords[1], bins = gridSize, 
                                range=[[minCoord1, maxCoord1], [minCoord2, maxCoord2]], weights = coordsOfInterest['vList'])
    
    splatted_image = gaussian_filter(grid, sigma = blur)
    
    plt.imshow(splatted_image.T, origin='lower', cmap='inferno', 
               extent=[minCoord1, maxCoord1, minCoord2, maxCoord2])
    plt.colorbar(label='Accumulated Value')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(f'Splatting along {axis}-axis, {opt2}')
    plt.show()

In [23]:
def plotCenterline(fileIn, cloud, center, opt2 = "Ia_filt"):
	coordsOfInterest = createCoordsOfInterest(fileIn, 3, opt2)
	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	for key, value in coordsOfInterest.items():
		coordsOfInterest[key] = value.flatten()
	
	if (cloud == "Inferno"):
		xmask = (coordsOfInterest['xList'] >= -20) & (coordsOfInterest['xList'] <= 0)
		for key, value in coordsOfInterest.items():
				coordsOfInterest[key] = value[xmask]
		ymask = (coordsOfInterest['yList'] >= -10) & (coordsOfInterest['yList'] <= 10)
		for key, value in coordsOfInterest.items():
				coordsOfInterest[key] = value[ymask]
		zmask = (coordsOfInterest['zList'] >= 0) & (coordsOfInterest['zList'] <= 20)
		for key, value in coordsOfInterest.items():
				coordsOfInterest[key] = value[zmask]
	elif (cloud == "Mushroom"):
		xmask = (coordsOfInterest['xList'] >= -10) & (coordsOfInterest['xList'] <= 0)
		for key, value in coordsOfInterest.items():
				coordsOfInterest[key] = value[xmask]
		ymask = (coordsOfInterest['yList'] >= 0) & (coordsOfInterest['yList'] <= 10)
		for key, value in coordsOfInterest.items():
				coordsOfInterest[key] = value[ymask]
		zmask = (coordsOfInterest['zList'] >= 0) & (coordsOfInterest['zList'] <= 15)
		for key, value in coordsOfInterest.items():
				coordsOfInterest[key] = value[zmask]
	else:
		print("Invalid cloud input")

	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']
	z = coordsOfInterest['zList']
	values = coordsOfInterest['vList']
	values[values == 0] = 10e-10
	values = np.log10(values)
	min = -1
	max = -9

	fig = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'thermal',
		isomin = min,
		isomax = max,
		surface_count = 10,
		opacity = 0.1,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	z_levels = coordsOfInterest['vList'].shape[2]
	maxPoints = []
	
	if (center == "max"):
		for z in range(z_levels):
			maxValue = np.max(coordsOfInterest['vList'][:, :, z])
			maxIndex = np.argmax(coordsOfInterest['vList'][:, :, z])
			x, y = np.unravel_index(maxIndex, coordsOfInterest['vList'][:, :, z].shape)
			xVal = coordsOfInterest['xList'][x, y, z]
			yVal = coordsOfInterest['yList'][x, y, z]
			zVal = coordsOfInterest['zList'][x, y, z]
			maxPoints.append((xVal, yVal, zVal, maxValue))

			x, y, z, v = zip(*maxPoints)
			line = go.Figure(data = go.Scatter3d(
				x=x, 
				y=y, 
				z=z,
				mode='lines+markers',
				line=dict(color='blue', width=1),
				marker=dict(size=1)
			))
	elif (center == "geom"):
		for z in range(z_levels):
			x_slice = coordsOfInterest['xList'][:, :, z]
			y_slice = coordsOfInterest['yList'][:, :, z]
			v_slice = coordsOfInterest['vList'][:, :, z]
			
			v_sum = np.sum(v_slice)
			if v_sum == 0:
				continue
			
			weighted_mean_x = np.sum(x_slice * v_slice) / v_sum
			weighted_mean_y = np.sum(y_slice * v_slice) / v_sum
			
			z_value = coordsOfInterest['zList'][0, 0, z]
			
			maxPoints.append((weighted_mean_x, weighted_mean_y, z_value))

		x, y, z = zip(*maxPoints)
		line = go.Figure(data = go.Scatter3d(
			x=x, 
			y=y, 
			z=z,
			mode='lines+markers',
			line=dict(color='blue', width=2),
			marker=dict(size=1)
		))

	figFinal = go.Figure(data = fig.data + line.data)

	figFinal.update_layout(
		title=f'Inferno (Logarithmic scale, {'Ia_filt'})',
		scene=dict(
			xaxis_title='East of COVIS (m)',
			yaxis_title='North of COVIS (m)',
			zaxis_title='Z Axis'
		),
		coloraxis_colorbar=dict(
			title=f'{'Ia_filt'} Values',
			tickvals=np.linspace(min, max, num=5),
			ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)	

	figFinal.show()

In [ ]:
plotCenterline(r'matFiles\COVIS-20230101T000002-imaging1.mat', "Inferno", "geom")